## Function to create a list with all the file paths within a path

In [654]:
def getFilePaths(path):
    # Function to return a list of files in a path:
    # Initialize a list for all the file names
    files = list()
    # Look for folders in the list
    for file in os.listdir(path):
        # Concatenate a path for each file
        pathfile = os.path.join(path, file)
        # Validate if the pathfile is a folder
        if os.path.isdir(pathfile):
            files = files + getFilePaths(pathfile)
        else:
            files.append(pathfile)
    return files      

## Create a dataframe with the list of paths

In [655]:
# Import dependencies
import pandas as pd, time, hashlib

In [656]:
# Path with the files to look for duplicates
path='../Photos copy'
# Initialize and empty dataframe with the paths of the files in the given path
files_df = pd.DataFrame(getFilePaths(path),columns=['path'])
# Initialize Columns for the dataframe 
files_df['hash'] = 0
files_df['filename'] = ''
files_df['count'] = 1
files_df['created'] = 0
files_df['modified'] = 0

# Loop to fill the dataframe columns with values
for row in range(len(files_df)):
    # Load the file
    image_file = open(files_df['path'][row],'rb').read()
    # Encode the file to a hexagsimal number
    files_df['hash'][row] = hashlib.md5(image_file).hexdigest()
    # Get the filename of the file
    files_df['filename'][row] = os.path.basename(files_df['path'][row])
    # Get the created date 
    files_df['created'][row] = pd.to_datetime(time.ctime(os.path.getctime(files_df['path'][row])))
    # Get the modified date 
    files_df['modified'][row] = pd.to_datetime(time.ctime(os.path.getmtime(files_df['path'][row])))
# Print first rows of the dataframe    
files_df.head()

,path,hash,filename,count,created,modified
0,../Photos copy/.DS_Store,5f27466e92caf57a930058b683ad5f7e,.DS_Store,1,2020-04-09 23:53:43,2020-04-09 18:38:53
1,../Photos copy/New Folder With Items 5/DSC_003...,b8cc4ead5b44bdd8e994c21287e618e8,DSC_0030.jpg,1,2020-04-09 23:53:43,2020-04-07 20:02:04
2,../Photos copy/New Folder With Items 5/DSC_002...,683d7efc80e910a2691880ca0ad9d059,DSC_0024.jpg,1,2020-04-09 23:53:43,2020-04-07 19:59:50
3,../Photos copy/New Folder With Items 5/DSC_001...,689a4e61ff572cb10ca471fb58004df1,DSC_0018.jpg,1,2020-04-09 23:53:43,2020-04-07 19:54:48
4,../Photos copy/New Folder With Items 5/DSC_001...,4d8c40e515d43b15a2dd6f826fdd1163,DSC_0019.jpg,1,2020-04-09 23:53:43,2020-04-07 19:55:54


In [657]:
import numpy as np, datetime
reapeated_files_df = files_df.groupby('hash').sum()
reapeated_files_df = reapeated_files_df[reapeated_files_df['count'] > 1]
pivot_df = pd.pivot_table(reapeated_files_df, values='count', index=['hash'], aggfunc=np.sum, margins=True)
now_string = datetime.datetime.now().strftime("%Y-%m-%d %H-%M-%S")
pivot_df.to_csv('repeated_' + now_string + '.csv' )
pivot_df

,count
hash,
All,0


In [658]:
if pivot_df['count']['All'] ==0:
    print('No repeated files were found')

else:

    repeated_path_df = files_df[files_df['hash'].isin(pivot_df.index.to_list())]
    repeated_path_df = repeated_path_df.groupby(['hash','path','created','modified']).sum().sort_values(by=['hash', 'created',                                                          'modified'], ascending=True)
    repeated_path_df.to_csv('repeated_list_' + now_string + '.csv' )


    repeated_path_df['action'] = ''
    repeated_path_df['action'][0] = 'leave' 


    for i in range(1,len(repeated_path_df.index.get_level_values(1).tolist())):
        if repeated_path_df.index.get_level_values(0)[i] == repeated_path_df.index.get_level_values(0)[i-1]:
            repeated_path_df['action'][i] = 'delete' 
        else:
            repeated_path_df['action'][i] = 'delete' 

    repeated_path_df     

No repeated files were found


In [659]:
if pivot_df['count']['All'] ==0:
    print('No repeated files were found')

else:
    for i in range(0,len(repeated_path_df.index.get_level_values(1).tolist())):
        if repeated_path_df['action'][i] == 'delete':
            #os.remove(repeated_path_df.index.get_level_values(1).tolist()[i])
 
    print('All duplicated images had been deleted')


IndentationError: expected an indented block (<ipython-input-659-74065fc78d6b>, line 5)